In [1]:
import pandas as pd
import numpy as np
import os 

In [2]:
os.getcwd()

'/Users/mayurbansal'

In [3]:
os.chdir('/Users/mayurbansal/documents/github/Dotin-Columbia-Castone-Team-Alpha-/data')

In [4]:
#read data
mouse=pd.read_csv('mouse_new_var.csv')

In [5]:
mouse.head()

,Index,user_id,action,cord_x,cord_y,distance,radio,time_since,id,created_at,updated_at,timeElapsed,window_x,window_y,system
0,0,365,s,0,0.0,0.000000,NaN,0.000,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860,pc
1,1,365,m,601,167.0,623.770791,NaN,0.220,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860,pc
2,2,365,m,602,166.0,1.414214,NaN,0.304,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860,pc
3,3,365,c,602,166.0,0.000000,NaN,0.432,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860,pc
4,4,365,m,602,166.0,0.000000,NaN,0.518,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860,pc


In [6]:
#Create two new features to capture x,y coordinates but improved, to account for variability in window size

#ratio of cord_x / window_x * 100 - % of width
mouse['percent_cord_x']=(mouse['cord_x']/mouse['window_x'])*100

#ratio of cord_y / window_y * 100 - % of length
mouse['percent_cord_y']=(mouse['cord_y']/mouse['window_y'])*100

In [7]:
#create a column to note the change in successvie x ordinates (in percentage of screen: width) 
mouse['shifted_percx'] = mouse.groupby(['user_id'])['percent_cord_x'].transform(lambda x:x-x.shift())

In [8]:
#create a column to note the change in successvie y ordinates (in percentage of screen: height) 
mouse['shifted_percy'] = mouse.groupby(['user_id'])['percent_cord_y'].transform(lambda x:x-x.shift())

In [9]:
#absolute value of shift to include the distance even if the user goes in opposite x direction
mouse['shifted_percx_absolute']=mouse['shifted_percx'].abs()

In [10]:
#absolute value of shift to include the distance even if the user goes in opposite y direction
mouse['shifted_percy_absolute']=mouse['shifted_percy'].abs()

In [11]:
mouse.head()

,Index,user_id,action,cord_x,cord_y,distance,radio,time_since,id,created_at,...,timeElapsed,window_x,window_y,system,percent_cord_x,percent_cord_y,shifted_percx,shifted_percy,shifted_percx_absolute,shifted_percy_absolute
0,0,365,s,0,0.0,0.000000,NaN,0.000,1,2019-03-05 06:56:35.848652,...,245.925,1600,860,pc,0.0000,0.000000,NaN,NaN,NaN,NaN
1,1,365,m,601,167.0,623.770791,NaN,0.220,1,2019-03-05 06:56:35.848652,...,245.925,1600,860,pc,37.5625,19.418605,37.5625,19.418605,37.5625,19.418605
2,2,365,m,602,166.0,1.414214,NaN,0.304,1,2019-03-05 06:56:35.848652,...,245.925,1600,860,pc,37.6250,19.302326,0.0625,-0.116279,0.0625,0.116279
3,3,365,c,602,166.0,0.000000,NaN,0.432,1,2019-03-05 06:56:35.848652,...,245.925,1600,860,pc,37.6250,19.302326,0.0000,0.000000,0.0000,0.000000
4,4,365,m,602,166.0,0.000000,NaN,0.518,1,2019-03-05 06:56:35.848652,...,245.925,1600,860,pc,37.6250,19.302326,0.0000,0.000000,0.0000,0.000000


In [12]:
#totalwidthcoveredbyuser
totalwidthcovered=mouse.groupby(['user_id'])['shifted_percx_absolute'].sum()

In [13]:
#totalheightcoveredbyuser
totalheightcovered=mouse.groupby(['user_id'])['shifted_percy_absolute'].sum()

In [14]:
#creating a new column to denote the width covered by user
width=pd.DataFrame(totalwidthcovered)

In [15]:
width['user_id']=width.index

In [16]:
width=width.reset_index(drop=True)

In [ ]:
#

In [17]:
width.head()

,shifted_percx_absolute,user_id
0,2772.125000,365
1,1565.407447,371
2,3455.051245,373
3,5897.364568,374
4,5423.572474,375


In [18]:
#creating a new column to denote the height covered by user
height=pd.DataFrame(totalheightcovered)

In [19]:
height['user_id']=height.index

In [20]:
height=height.reset_index(drop=True)

In [21]:
height.head()

,shifted_percy_absolute,user_id
0,3744.767442,365
1,2256.410256,371
2,4327.609890,373
3,6338.186813,374
4,5956.181319,375


In [22]:
#merging these columns together in a dataframe
mouse_user=pd.merge(width,height)

In [23]:
mouse_user

,shifted_percx_absolute,user_id,shifted_percy_absolute
0,2772.125000,365,3744.767442
1,1565.407447,371,2256.410256
2,3455.051245,373,4327.609890
3,5897.364568,374,6338.186813
4,5423.572474,375,5956.181319
...,...,...,...
725,10448.975110,1834,5575.203252
726,9596.925329,1835,8529.395604
727,3158.125915,1836,4329.395604
728,4295.312500,1837,4356.263736


In [24]:
#measure of screen covered 
mouse_user['measure_width_covered']=mouse_user['shifted_percx_absolute']/100
mouse_user['measure_height_covered']=mouse_user['shifted_percy_absolute']/100

In [25]:
mouse_user.head()

,shifted_percx_absolute,user_id,shifted_percy_absolute,measure_width_covered,measure_height_covered
0,2772.125000,365,3744.767442,27.721250,37.447674
1,1565.407447,371,2256.410256,15.654074,22.564103
2,3455.051245,373,4327.609890,34.550512,43.276099
3,5897.364568,374,6338.186813,58.973646,63.381868
4,5423.572474,375,5956.181319,54.235725,59.561813


In [26]:
##working with directions of user movements

In [27]:
mouse.head()

,Index,user_id,action,cord_x,cord_y,distance,radio,time_since,id,created_at,...,timeElapsed,window_x,window_y,system,percent_cord_x,percent_cord_y,shifted_percx,shifted_percy,shifted_percx_absolute,shifted_percy_absolute
0,0,365,s,0,0.0,0.000000,NaN,0.000,1,2019-03-05 06:56:35.848652,...,245.925,1600,860,pc,0.0000,0.000000,NaN,NaN,NaN,NaN
1,1,365,m,601,167.0,623.770791,NaN,0.220,1,2019-03-05 06:56:35.848652,...,245.925,1600,860,pc,37.5625,19.418605,37.5625,19.418605,37.5625,19.418605
2,2,365,m,602,166.0,1.414214,NaN,0.304,1,2019-03-05 06:56:35.848652,...,245.925,1600,860,pc,37.6250,19.302326,0.0625,-0.116279,0.0625,0.116279
3,3,365,c,602,166.0,0.000000,NaN,0.432,1,2019-03-05 06:56:35.848652,...,245.925,1600,860,pc,37.6250,19.302326,0.0000,0.000000,0.0000,0.000000
4,4,365,m,602,166.0,0.000000,NaN,0.518,1,2019-03-05 06:56:35.848652,...,245.925,1600,860,pc,37.6250,19.302326,0.0000,0.000000,0.0000,0.000000


In [28]:
#counting how many times each user moves to left/right or stays at the same position
mouse_direction_x=mouse.groupby('user_id').shifted_percx.apply(lambda x: pd.Series([(x < 0).sum(), (x > 0).sum(),(x == 0).sum() ])).unstack()

In [29]:
mouse_direction_x=pd.DataFrame(mouse_direction_x)

In [30]:
mouse_direction_x

,0,1,2
user_id,,,
365,1427,1332,2245
371,1681,1557,805
373,2347,2175,1429
374,1829,1546,1599
375,1447,1459,998
...,...,...,...
1834,6842,7347,3771
1835,6402,7502,4101
1836,2033,1827,1324


### here 0 means right to left , 1 means left to right , 2 means no movement horizontally

In [31]:
#adding users as column
mouse_direction_x['user_id']=mouse_direction_x.index

In [32]:
#drop index to avoid confusion
mouse_direction_x=mouse_direction_x.reset_index(drop=True)

In [33]:
mouse_direction_x=mouse_direction_x.rename(columns={0:"movesleft", 1:"movesright", 2:"no_horizontal_movement"})


In [34]:
mouse_direction_x.head()

,movesleft,movesright,no_horizontal_movement,user_id
0,1427,1332,2245,365
1,1681,1557,805,371
2,2347,2175,1429,373
3,1829,1546,1599,374
4,1447,1459,998,375


In [35]:
#adding a column calculating the total horizontal movements per user
mouse_direction_x['totalmovements_x'] = mouse_direction_x['movesleft']+mouse_direction_x['movesright']+mouse_direction_x['no_horizontal_movement']


In [36]:
#calculating percentage of movements in each direction
mouse_direction_x['perc_left_movement']=(mouse_direction_x['movesleft']/mouse_direction_x['totalmovements_x'])*100
mouse_direction_x['perc_right_movement']=(mouse_direction_x['movesright']/mouse_direction_x['totalmovements_x'])*100
mouse_direction_x['perc_no_movement_x']=(mouse_direction_x['no_horizontal_movement']/mouse_direction_x['totalmovements_x'])*100

In [37]:
mouse_direction_x.head()

,movesleft,movesright,no_horizontal_movement,user_id,totalmovements_x,perc_left_movement,perc_right_movement,perc_no_movement_x
0,1427,1332,2245,365,5004,28.517186,26.618705,44.864109
1,1681,1557,805,371,4043,41.578036,38.511007,19.910957
2,2347,2175,1429,373,5951,39.438750,36.548479,24.012771
3,1829,1546,1599,374,4974,36.771210,31.081624,32.147165
4,1447,1459,998,375,3904,37.064549,37.371926,25.563525


In [38]:
##counting how many times each user moves up/down or stays at the same position
mouse_direction_y=mouse.groupby('user_id').shifted_percy.apply(lambda x: pd.Series([(x < 0).sum(), (x > 0).sum(),(x == 0).sum() ])).unstack()

In [39]:
mouse_direction_y=pd.DataFrame(mouse_direction_y)

In [40]:
mouse_direction_y['user_id']=mouse_direction_y.index

In [41]:
mouse_direction_y=mouse_direction_y.reset_index(drop=True)

In [42]:
mouse_direction_y.head()

,0,1,2,user_id
0,1512,2620,872,365
1,1327,1950,766,371
2,1719,2539,1693,373
3,1626,2169,1179,374
4,1718,1476,710,375


### here 0 means upward movement, 1 means downwards, 2 means no vertical movement

In [43]:
mouse_direction_y=mouse_direction_y.rename(columns={0:"movesup", 1:"movesdown", 2:"no_vertical_movement"})


In [44]:
mouse_direction_y['totalmovements_y'] = mouse_direction_y['movesup']+mouse_direction_y['movesdown']+mouse_direction_y['no_vertical_movement']


In [45]:
mouse_direction_y['perc_upwward_movement']=(mouse_direction_y['movesup']/mouse_direction_y['totalmovements_y'])*100
mouse_direction_y['perc_downward_movement']=(mouse_direction_y['movesdown']/mouse_direction_y['totalmovements_y'])*100
mouse_direction_y['perc_no_movement_y']=(mouse_direction_y['no_vertical_movement']/mouse_direction_y['totalmovements_y'])*100



In [46]:
mouse_direction_y.head()

,movesup,movesdown,no_vertical_movement,user_id,totalmovements_y,perc_upwward_movement,perc_downward_movement,perc_no_movement_y
0,1512,2620,872,365,5004,30.215827,52.358114,17.426059
1,1327,1950,766,371,4043,32.822162,48.231511,18.946327
2,1719,2539,1693,373,5951,28.885902,42.665098,28.449000
3,1626,2169,1179,374,4974,32.689988,43.606755,23.703257
4,1718,1476,710,375,3904,44.006148,37.807377,18.186475


In [47]:
mouse_direction_x.describe()

,movesleft,movesright,no_horizontal_movement,user_id,totalmovements_x,perc_left_movement,perc_right_movement,perc_no_movement_x
count,730.000000,730.000000,730.000000,730.000000,730.000000,728.000000,728.000000,728.000000
mean,3356.897260,3404.710959,4334.194521,1231.353425,11095.802740,32.450831,32.831808,34.717361
std,2270.658432,2338.705505,11269.120226,428.055521,13855.928132,9.873908,9.771801,18.816877
min,0.000000,0.000000,0.000000,365.000000,0.000000,2.982675,3.286301,0.000000
25%,2001.250000,2017.500000,1899.750000,894.750000,6125.000000,30.723906,31.149087,23.732693
50%,3183.500000,3200.000000,2624.000000,1288.500000,9109.500000,35.117027,35.574949,28.367935
75%,4420.000000,4444.500000,3514.750000,1605.750000,12762.000000,38.454150,38.766714,36.337893
max,19213.000000,17994.000000,207425.000000,1838.000000,240324.000000,75.000000,61.635220,93.731023


In [48]:
mouse_direction_y.describe()

,movesup,movesdown,no_vertical_movement,user_id,totalmovements_y,perc_upwward_movement,perc_downward_movement,perc_no_movement_y
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,2691.883562,3867.057534,4536.908219,1231.353425,11095.849315,28.811827,41.094060,30.094113
std,1681.045484,2179.015282,11243.388385,428.055521,13855.890820,9.314218,10.330092,14.205520
min,5.000000,0.000000,0.000000,365.000000,8.000000,4.961811,0.000000,0.000000
25%,1763.000000,2696.250000,1291.750000,894.750000,6125.000000,23.099064,35.647251,21.270255
50%,2499.000000,3911.000000,2320.500000,1288.500000,9109.500000,28.369195,41.581081,25.996271
75%,3388.250000,4921.500000,4554.500000,1605.750000,12762.000000,33.467780,47.435384,37.154251
max,16049.000000,20338.000000,204970.000000,1838.000000,240324.000000,100.000000,83.626302,85.289026


In [51]:
mouse_direction=pd.merge(mouse_direction_x,mouse_direction_y)

In [52]:
mouse_direction.head()

,movesleft,movesright,no_horizontal_movement,user_id,totalmovements_x,perc_left_movement,perc_right_movement,perc_no_movement_x,movesup,movesdown,no_vertical_movement,totalmovements_y,perc_upwward_movement,perc_downward_movement,perc_no_movement_y
0,1427,1332,2245,365,5004,28.517186,26.618705,44.864109,1512,2620,872,5004,30.215827,52.358114,17.426059
1,1681,1557,805,371,4043,41.578036,38.511007,19.910957,1327,1950,766,4043,32.822162,48.231511,18.946327
2,2347,2175,1429,373,5951,39.438750,36.548479,24.012771,1719,2539,1693,5951,28.885902,42.665098,28.449000
3,1829,1546,1599,374,4974,36.771210,31.081624,32.147165,1626,2169,1179,4974,32.689988,43.606755,23.703257
4,1447,1459,998,375,3904,37.064549,37.371926,25.563525,1718,1476,710,3904,44.006148,37.807377,18.186475


In [53]:
mouse_user_direction=pd.merge(mouse_user,mouse_direction)

In [54]:
mouse_user_direction.head()

,shifted_percx_absolute,user_id,shifted_percy_absolute,measure_width_covered,measure_height_covered,movesleft,movesright,no_horizontal_movement,totalmovements_x,perc_left_movement,perc_right_movement,perc_no_movement_x,movesup,movesdown,no_vertical_movement,totalmovements_y,perc_upwward_movement,perc_downward_movement,perc_no_movement_y
0,2772.125000,365,3744.767442,27.721250,37.447674,1427,1332,2245,5004,28.517186,26.618705,44.864109,1512,2620,872,5004,30.215827,52.358114,17.426059
1,1565.407447,371,2256.410256,15.654074,22.564103,1681,1557,805,4043,41.578036,38.511007,19.910957,1327,1950,766,4043,32.822162,48.231511,18.946327
2,3455.051245,373,4327.609890,34.550512,43.276099,2347,2175,1429,5951,39.438750,36.548479,24.012771,1719,2539,1693,5951,28.885902,42.665098,28.449000
3,5897.364568,374,6338.186813,58.973646,63.381868,1829,1546,1599,4974,36.771210,31.081624,32.147165,1626,2169,1179,4974,32.689988,43.606755,23.703257
4,5423.572474,375,5956.181319,54.235725,59.561813,1447,1459,998,3904,37.064549,37.371926,25.563525,1718,1476,710,3904,44.006148,37.807377,18.186475
